# Retinal Organoid and Stim Data
#### Reads digital file of stim, reads neuron timestamps, merges data, imprt them to neuroexplorer, gets neuroexplorer output, categorizes groups and plots the data

#### Step 01 _ Read txt file of stim Digit file and extract On and OFF times in seconds

##### The Whole code to: 
- Read txt files of digital bit files,  
- Convert to excel 
- Splits Stim time points to applied Freq and pulse width (each sheet contains specific Freq and pulse width)

In [ ]:
## Step 01 _ Read txt file of stim Digit file and extract the pulse-start and pulse-end times
import os
import pandas as pd

def extract_stimulus_events(txt_file):
    stim_on = []  # List to store stimulus onset times
    stim_off = []  # List to store stimulus offset times
    
    with open(txt_file, 'r') as file:
        lines = file.readlines()
        lines = lines[2:]
        
        for i in range(1, len(lines)):  # Start from the second row
            try:
                prev_value = int(lines[i-1].split()[1])
                curr_value = int(lines[i].split()[1])
                if prev_value == 0 and curr_value == 4:
                    stim_on.append(float(lines[i].split()[0]) / 1000)  # Convert to seconds
                elif prev_value == 4 and curr_value == 0:
                    stim_off.append(float(lines[i].split()[0]) / 1000)  # Convert to seconds
            except (ValueError, IndexError):
                continue  
    
    return stim_on, stim_off

def export_to_excel(txt_file, stim_on, stim_off):
    file_name = os.path.splitext(txt_file)[0]
    df = pd.DataFrame({'Stim_ON': stim_on, 'Stim_OFF': stim_off})
    output_file = f"{file_name}_stimulus_times.xlsx"
    df.to_excel(output_file, index=False)
    return output_file

def split_data_and_save_to_excel(input_file, output_file):
    df = pd.read_excel(input_file)
    categories = ['50ms_0.2Hz', '50ms_1.0Hz', '50ms_2.0Hz', '50ms_5.0Hz']
    start_indices = [2, 7, 17, 37] # This depends on light stim protocol (e.g. pulse 2 to 6 is 50ms_0.5Hz)
    end_indices = [6, 16, 36, 61]  
    
    with pd.ExcelWriter(output_file) as writer:
        for cat, start, end in zip(categories, start_indices, end_indices):
            cat_df = df.iloc[start-2:end-1]  
            cat_df.to_excel(writer, sheet_name=cat, index=False)

def process_files_and_split_data(txt_folder):
    for txt_file in os.listdir(txt_folder):
        if txt_file.endswith(".txt"):
            txt_path = os.path.join(txt_folder, txt_file)
            stim_on, stim_off = extract_stimulus_events(txt_path)
            excel_file = export_to_excel(txt_path, stim_on, stim_off)
            split_data_and_save_to_excel(excel_file, f"{os.path.splitext(txt_path)[0]}_splitted.xlsx")
            print(f"Processed file: {txt_file}")

# Example usage:
txt_folder = r'C:\StimTimes_Digital_Txt' 
process_files_and_split_data(txt_folder)

#### Step 02 _ Merge Stim ON-OFF file with .CSV of Unit time series

- Merges the .CSV file of unit time series with .xlsx file of StimTimestamps_Splitted (from previous step).
- Cosider that stimtimestamps are in seconds but neuron timestamps still not. this will be fixed in next steps

In [ ]:
## Merge Pulse timestamps with neuronal timestamp file
import pandas as pd
import os

def extract_data_from_excel(excel_file):
    sheets_data = pd.read_excel(excel_file, sheet_name=None)
    
    extracted_data = {}
    for sheet_name, sheet_df in sheets_data.items():
        extracted_data[sheet_name] = sheet_df.iloc[:, :2]
    return extracted_data

def add_sheet_names_to_columns(data):
    for sheet_name, sheet_data in data.items():
        new_column_names = [f"{sheet_name}_{col}" for col in sheet_data.columns]
        sheet_data.columns = new_column_names
    return data

def merge_excel_and_csv(excel_data, csv_file):
    csv_data = pd.read_csv(csv_file)
    merged_data = pd.concat([pd.concat(excel_data.values(), axis=1), csv_data], axis=1)
    return merged_data

def save_to_csv(output_csv_file, merged_data):
    merged_data.to_csv(output_csv_file, index=False)

# Load pulse timestamp file and unit timestamp file
excel_folder = r"C:\StimTimestamps" 
csv_folder = r"C:\UnitTimeseries"  
output_folder = r"C:\Stim-Unit_Timestamps"  
os.makedirs(output_folder, exist_ok=True)

for excel_file in os.listdir(excel_folder):
    if excel_file.endswith(".xlsx"):
        excel_path = os.path.join(excel_folder, excel_file)
        extracted_data = extract_data_from_excel(excel_path)
        add_sheet_names_to_columns(extracted_data)
        
        for csv_file in os.listdir(csv_folder):
            if csv_file.endswith(".csv") and excel_file[:28] in csv_file:
                csv_path = os.path.join(csv_folder, csv_file)
                merged_data = merge_excel_and_csv(extracted_data, csv_path)
                output_csv_file_name = os.path.splitext(csv_file)[0] + "_merged.csv"
                output_csv_file_path = os.path.join(output_folder, output_csv_file_name)
                save_to_csv(output_csv_file_path, merged_data)

#### Step 03 _ Make data ready for NEX by multiplying stim times in sampling frequency

- In this code It reads all Stim_timestamps from Merged file and multiply them with Sampling Frequency. 
- It also removes the index column from column 15
- The rest of dataset is returned as it is
- It is required for loading the data into Neuroexplorer

In [ ]:
## Prepare data to for importing to Neuroexplorer (Nex). Later these data will be used to extract PSTH data

import os
import pandas as pd

def adjust_sampling_frequency(csv_file, sampling_frequency):
    # Read the CSV file
    df = pd.read_csv(csv_file)
    df.iloc[0:, 0:14] *= sampling_frequency #Stimtimes*samplingFreq
    df.drop(columns=df.columns[14], inplace=True)
    return df

def process_files_and_save_to_csv(csv_folder, output_folder, sampling_frequency):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith(".csv"):
            csv_path = os.path.join(csv_folder, csv_file)
            df_adjusted = adjust_sampling_frequency(csv_path, sampling_frequency)
            output_file_name = os.path.splitext(csv_file)[0] + "_stimXsampFreq.csv"
            output_file = os.path.join(output_folder, output_file_name)
            df_adjusted.to_csv(output_file, index=False)

csv_folder = r"C:\Merged"  
output_folder = r"C:\Merged_StimXsampFreq"  
sampling_frequency = 32000  # Specify the sampling frequency

process_files_and_save_to_csv(csv_folder, output_folder, sampling_frequency)

##################################################################################################
##################################################################################################
##################################################################################################
Merged_StimXsampFreq is imported to NEX to extract the Burst, PSTH, RateHisto data, .....
Automated NEX scripts runs a template algorithems (adjusted by user) to extract PSTH, Burst, .... data
##################################################################################################
##################################################################################################
##################################################################################################

#### Step 04 Example NexCode for extracting PSTH, Burst, RateHisto from many files (each file is data of one sample in one specific day)

% This is a neuroexplorer script ( .nsc ) 
% which reads many excel files containing neurons and their timestamps and stimulation time stamps as well and extracts PSTH data
% read All files in one folder, run analysis, export data for each MEA in a separate file and for each week in a separate sheet
% After generation of templates for each variable , copy and paste the code in the neuroexplorer script, define directory, run the code in NEX

SourcePath="C:\folder\"     % Enter the directory with the source data mcd or nex. Do NOT FORGET a backslash at the end!
SourceFilter = SourcePath + "*.csv"             % Process files with given extension (e.g., .nex or .mcd) only

FileCount = GetFileCount(SourceFilter)
Trace(FileCount, " files to analyze")           

for i= 1 to FileCount
	name = GetFileName(i)
	Trace(name)                                 % prints "the name" of the all files ready to be analysed
	doc = OpenDocument(name)
	Title = GetDocTitle(doc)
	
    if doc > 0
        SelectAllNeurons(doc)                                                                
        %ApplyTemplate(doc, "Burst__definedbyuser_01")        % Burst detection template has already been generated and saved in template folder of NEX
        ApplyTemplate(doc, "PSTH_Kritika__definedbyuser_01")  % PSTH template has already been generated and saved in template folder of NEX  
        %ApplyTemplate(doc, "RateHistogram_definedbyuser_01") % Rate Histo template has already been generated and saved in template folder of NEX              
        
        ExcelFileName= "PSTH"+ Left(Title,1)    % the name of the output file in each loop 
        Trace(ExcelFileName)                    % prints the MEA for which the data is extracting
        ExcelSheetName= Left (Title,25)         % the name of the output sheet in each loop
        Trace(ExcelSheetName)                   % prints the MEA for which the data is extracting
        
        % save all data
        %SendResultsToExcel(doc,"C:\PSTH_data_01.xlsx", ExcelSheetName, 0, "A1", 1, 1)
        % save only Summary 
        SendResultsSummaryToExcel(doc,"C:\PSTH_data_02.xlsx", ExcelSheetName, 0, "A1", 1, 1) 
        CloseDocument(doc)                      % prevents from accumulation of open files in Nex
     end
end

#### Step 05 xlsx output of Nex data is loaded in following codes for further categorization and analysis

In [ ]:
# Find the details of light stimulation protocol in the 'Variable' rows and extract average data of each protocol for all neurons

import pandas as pd
import os

## Read the Excel file (ouput of Nex for PSTH)
input_excel_file = r"C:\PSTH_data.xlsx"
xls = pd.ExcelFile(input_excel_file)
input_directory = os.path.dirname(input_excel_file)
output_excel_file = os.path.join(input_directory, 'PSTH_Processed.xlsx')

first_sheet_name = xls.sheet_names[0]
df_first_sheet = pd.read_excel(xls, first_sheet_name, header=0) 
variable_names = df_first_sheet.columns.tolist()
print(f'variable_names', variable_names)

processed_sheets = {}
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name)
    if df.empty:
        print(f"Sheet '{sheet_name}' is empty. Skipping.")
        continue
    filtered_data = df[~df['Variable'].str.startswith(('50ms'))].copy()
    filtered_data['Neuron'] = filtered_data['Variable'].apply(lambda x: x.split(' vs. ')[0].split()[0])
    filtered_data['Protocol'] = filtered_data['Variable'].apply(lambda x: x.split(' vs. ')[1])
    filtered_data['Pulse_Width'] = filtered_data['Protocol'].apply(lambda x: x.split('_')[0])
    filtered_data['Frequency'] = filtered_data['Protocol'].apply(lambda x: x.split('_')[1].split('Hz')[0])
    
    # Group by protocol and calculate averages for desired variables
    grouped_data = filtered_data.groupby(['Protocol', 'Pulse_Width', 'Frequency']).agg({
        'NumRefEvents': 'mean',
        'Spikes': 'mean',
        'Filter Length': 'mean',
        'Mean Freq.': 'mean',
        'Mean Hist.': 'mean',
        'Z-score Mean': 'mean',
        'Peak Z-score': 'mean',
        'Peak/Mean': 'mean',
        'Peak Position': 'mean',
        'Peak Half Height': 'mean',
        'Peak Width at Half Height': 'mean'
    }).reset_index()

    processed_sheets[sheet_name] = grouped_data


with pd.ExcelWriter(output_excel_file) as writer:
    for sheet_name, processed_data in processed_sheets.items():
        processed_data.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Processed data saved to {output_excel_file}")

### Step 06 further categorization steps, preparing data for Graphpad prism, and plotting

In [ ]:
## adds a column to the each dataset column name=seet name (sample name)
import pandas as pd

def modify_sheet_names_and_add_sample_name(excel_file):
    # Read the Excel file with all sheets
    xls = pd.ExcelFile(excel_file)
    sheet_names = xls.sheet_names
    modified_sheet_names = [name.replace(".raw_m", "") for name in sheet_names]
    return xls, modified_sheet_names
def copy_data_to_new_excel_with_sample_name(xls, modified_sheet_names, output_excel):
    with pd.ExcelWriter(output_excel) as writer:
        for sheet_name, mod_sheet_name in zip(xls.sheet_names, modified_sheet_names):
            df = pd.read_excel(xls, sheet_name=sheet_name)
            df.insert(0, "Sample Name", sheet_name)
            df.to_excel(writer, sheet_name=mod_sheet_name, index=False)

excel_file = r"C:\PSTH_Processed.xlsx" 
output_excel = r"C:\PSTH_Processed_02.xlsx"  

xls, modified_names = modify_sheet_names_and_add_sample_name(excel_file)
copy_data_to_new_excel_with_sample_name(xls, modified_names, output_excel)

In [ ]:
## Concatinate all data
import pandas as pd
file_path = r"C:\PSTHMerged_Processed_02.xlsx"
xls = pd.ExcelFile(file_path)
suffixes = ['_02', '_03', '_04']
concatenated_sheets = {}
for suffix in suffixes:
    relevant_sheets = [sheet_name for sheet_name in xls.sheet_names if sheet_name.endswith(suffix)]
    concatenated_df = pd.concat([pd.read_excel(xls, sheet_name) for sheet_name in relevant_sheets])
    concatenated_sheets[f'Intensity{suffix}'] = concatenated_df

output_file_path = r"C:\ConcatPSTH.xlsx"
with pd.ExcelWriter(output_file_path) as writer:
    for sheet_name, df in concatenated_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
# Groupp samples based on Week
import pandas as pd

file_path = r"C:\ConcatPSTH.xlsx"
xls = pd.ExcelFile(file_path)
processed_sheets = {}
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name)
    df['Week'] = df['Sample Name'].str[:3]
    df['Group'] = df['Sample Name'].apply(lambda x: 'Vascularized' if 'VASCU' in x else 'Non_Vascularized')
    new_columns = ['Group', 'Week'] + [col for col in df.columns if col not in ['Group', 'Week']]
    df = df[new_columns]
    processed_sheets[sheet_name] = df
output_file_path = r"C:\Groupped_ConcatPSTH.xlsx"
with pd.ExcelWriter(output_file_path) as writer:
    for sheet_name, df in processed_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
## Plot data
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

colors = {'Non_Vascularized': 'gray', 'Vascularized': 'lightcoral'}

file_path = r"C:\Groupped_ConcatPSTH.xlsx"
xls = pd.ExcelFile(file_path)

## remove outliers
def remove_outliers(df, column):
    def _remove_outliers(group):
        Q1 = group[column].quantile(0.25)
        Q3 = group[column].quantile(0.75)
        IQR = Q3 - Q1
        return group[~((group[column] < (Q1 - 3 * IQR)) | (group[column] > (Q3 + 5 * IQR)))]
    return df.groupby(['Group', 'Week']).apply(_remove_outliers).reset_index(drop=True)


def calculate_group_statistics(df):
    group_stats = df.groupby(['Week', 'Group'])['Mean Hist.'].agg(['mean', 'std', 'sem', 'max', 'min', 'count']).reset_index()
    group_counts = df.groupby('Group')['Mean Hist.'].count().reset_index()
    group_stats['Count'] = group_counts['Mean Hist.']
    return group_stats

processed_data = {}
variable_stats_raw = {}
variable_stats_cleaned = {}
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name)
    df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['Week', 'Mean Hist.'])
    group_stats_raw = calculate_group_statistics(df)
    variable_stats_raw[sheet_name] = group_stats_raw
    
    # --------------------- Remove outliers--------------------
    df_cleaned = remove_outliers(df, 'Mean Hist.')
    processed_data[sheet_name] = df_cleaned
    group_stats = calculate_group_statistics(df_cleaned)
    variable_stats_cleaned[sheet_name] = group_stats
    group_order = ['Non_Vascularized', 'Vascularized']
    plt.figure(figsize=(7, 6))  
    ax=sns.boxplot(data=df_cleaned, x='Week', y='Mean Hist.', hue='Group', 
                order=['w17', 'w20', 'w23'],
                hue_order=group_order, palette=colors, showfliers=False)  
    sns.stripplot(data=df_cleaned, x='Week', y='Mean Hist.', hue='Group', 
                  order=['w17', 'w20', 'w23'],
                  hue_order=group_order, dodge=True, jitter=True, edgecolor='black', linewidth=0.5, palette=colors, 
                  alpha=0.7, size=4) 
    means = df_cleaned.groupby(['Week', 'Group'])['Mean Hist.'].mean().reset_index()
    for group, color in colors.items():
        plt.plot(means[means['Group'] == group]['Week'], means[means['Group'] == group]['Mean Hist.'], 
                 color=color, linestyle='-', linewidth=2, marker='o', markersize=5, markeredgecolor='black',
                 markeredgewidth=3)      
    for group, color in colors.items():
        sems = df_cleaned.groupby(['Week', 'Group'])['Mean Hist.'].sem().reset_index()
        group_sems = sems[sems['Group'] == group]
        for dpi in ['w17', 'w20', 'w23']:
            dpi_sems = group_sems[group_sems['Week'] == dpi]
            plt.errorbar(dpi_sems['Week'], means[(means['Group'] == group) & (means['Week'] == dpi)]['Mean Hist.'], 
                         yerr=dpi_sems['Mean Hist.'], fmt='o', color=color, markersize=10, capsize=5, 
                         markeredgecolor='black', linewidth=3)  
    
    plt.xticks(rotation=45, fontsize=18)
    ax.set_yscale('log')
    ax.set_ylim(1e-1, 1e3) 
    plt.legend(title='Group', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xlabel('Week', fontsize=16)  
    plt.ylabel('Mean Hist.', fontsize=18)  
    plt.title(f'{sheet_name}', fontsize=18)  
    plt.yticks(fontsize=16)  
    plt.tight_layout()
    plt.show()

output_excel_path = r'C:\Groupped_ConcatPSTH_Stat.xlsx'
with pd.ExcelWriter(output_excel_path) as writer:
    for variable, stats in variable_stats_raw.items():
        stats.to_excel(writer, sheet_name=f"{variable}_raw", index=False)
    for variable, stats in variable_stats_cleaned.items():
        stats.to_excel(writer, sheet_name=f"{variable}_clean", index=False)
output_excel_path_cleaned = r'C:\Groupped_ConcatPSTH_OutliersRemoved.xlsx'
with pd.ExcelWriter(output_excel_path_cleaned) as writer_cleaned:
    for sheet_name, data in processed_data.items():
        data.to_excel(writer_cleaned, sheet_name=sheet_name, index=False)    